# CNN으로 패션 아이템 구분하기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
EPOCHS     = 40
BATCH_SIZE = 64

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [7]:
model     = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.327074
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.304774
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.518547
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.639790
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.553578
[1] Test Loss: 0.2035, Accuracy: 93.88%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.583562
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.498570
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.317146
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.367570
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.330507
[2] Test Loss: 0.1260, Accuracy: 96.06%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.534283
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.356768
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.164879
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.300566
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.241633
[3] Test Loss: 0.0996, Accuracy: 96.84%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.297092
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.479451
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.222132


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.266177
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.078930
[29] Test Loss: 0.0342, Accuracy: 98.88%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.136740
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.110551
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.096758
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.050359
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.135727
[30] Test Loss: 0.0340, Accuracy: 98.93%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.139015
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.102659
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.109909
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.121773
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.064748
[31] Test Loss: 0.0365, Accuracy: 98.84%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.103884
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.132301
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.061583
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.097075
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.066446
[32] Test Loss: 0.0329, A